In [ ]:
import busio
import time
import datetime
import pandas as pd
from Adafruit_BNO055 import BNO055
import utils
import os
import collections
import joblib
import subprocess
import shlex
import numpy
from gpiozero import LED
from time import sleep

model = joblib.load('/home/pi/ayoTA/models/In19_391pt_model.joblib')

ledb=LED(20)
ledg=LED(16)
ledr=LED(12)

def turn_on(prediction):
    if prediction == 'dumbbell-curl' :
        ledr.on()
        sleep(2)
        ledr.off()
    if prediction == 'lateral-raise' :
        ledb.on()
        sleep(2)
        ledb.off()
    if prediction == 'military-press' :
        ledg.on()
        sleep(2)
        ledg.off()
    if prediction == 'seated-rows' :
        ledr.on()
        ledg.on()
        sleep(2)
        ledg.off()
        ledr.off()
    if prediction == 'skull-crusher' :
        ledb.on()
        ledg.on()
        sleep(2)
        ledb.off()
        ledg.off()
    if prediction == 'forwardpunch' :
        ledr.on()
        ledb.on()
        sleep(2)
        ledb.off()
        ledr.off()


def read_sensors(bno):
    vector = bno._read_vector(BNO055.BNO055_ACCEL_DATA_X_LSB_ADDR, 22)
    accel = [s / 100. for s in vector[:3]]
    mag = [s / 16. for s in vector[3:6]]
    gyro = [s / 900. for s in vector[6:9]]
    euler = [s / 16. for s in vector[9:12]]
    quaternion = [s / QUATERNION_SCALE for s in vector[12:16]]
    lin_accel = [s / 100. for s in vector[16:19]]
    gravity = [s / 100. for s in vector[19:22]]

    return accel + mag + gyro + euler + quaternion + lin_accel + gravity

SAMPLE_RATE_HZ = 100
QUATERNION_SCALE = (1.0 / (1 << 14))

CHECK_TIME_INCREMENT_MS = 1000
SAMPLE_SIZE_MS = 2000

bno = BNO055.BNO055(address=0x29)

# Initialize the BNO055 and stop if something went wrong.
if not bno.begin():
    raise RuntimeError('Failed to initialize BNO055! Is the sensor connected?')

# Print system status and self test result.
status, self_test, error = bno.get_system_status()
print('System status: {0}'.format(status))
print('Self test result (0x0F is normal): 0x{0:02X}'.format(self_test))

# Print out an error if system status is in error mode.
if status == 0x01:
    print('System error: {0}'.format(error))
    print('See datasheet section 4.3.59 for the meaning.')

i = 0
header = ["time_ms"] + utils.get_sensor_headers()
data = collections.deque(maxlen=int(SAMPLE_SIZE_MS / 10)) #10 Hz

print('Starting operation')

start = datetime.datetime.now()
elapsed_ms = 0
last_classified = 0
last_classification = "negative_trim"

while True:
    row = [elapsed_ms] + read_sensors(bno)
    data.append(row)

    if elapsed_ms - last_classified >= CHECK_TIME_INCREMENT_MS and len(data) == data.maxlen:
        df = pd.DataFrame(list(data), columns=header)
        features = utils.get_model_features(df)
        prediction = model.predict([features])[0]

        #print(int(elapsed_ms), prediction)
        if prediction != 'negative_trim' and last_classification != prediction :
            print("========================>", prediction)
            turn_on(prediction)
            data.clear()

        last_classified = elapsed_ms
        last_classification = prediction

    elapsed_ms = (datetime.datetime.now() - start).total_seconds() * 1000


System status: 133
Self test result (0x0F is normal): 0x0F
Starting operation
========================> military-press
========================> military-press
========================> seated-rows
========================> skull-crusher
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> military-press
========================> lateral-raise
========================>